In [ ]:
# Install required packages first:
%pip install fastapi uvicorn langchain langchain-groq langchain-core langchain-community pyngrok

In [ ]:


import os
from fastapi import FastAPI
from pydantic import BaseModel
from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# Set your Groq API key (you can also set via environment variable before running uvicorn)
os.environ["GROQ_API_KEY"] = "grok_api_key"

# Initialize FastAPI
app = FastAPI(title="Nirvaan AI Chatbot API")

# Initialize model
llm = ChatGroq(model="llama-3.1-8b-instant")

# Store conversation history per session (in-memory for now)
chat_history = [
    SystemMessage(
        content=(
            "You are a compassionate and supportive mental health chatbot designed for students. "
            "Your mission is to provide empathetic, safe, and multilingual psychological support for "
            "students dealing with stress, anxiety, exam pressure, loneliness, and emotional struggles. "
            "If the user asks about unrelated topics (like cricket, news, general knowledge etc.), "
            "gently remind them: 'I am here to support your mental wellbeing and student life challenges. "
            "Would you like to share how you are feeling?'"
        )
    )
]

# Define request/response schema
class ChatRequest(BaseModel):
    message: str

class ChatResponse(BaseModel):
    response: str

@app.post("/chat", response_model=ChatResponse)
def chat(request: ChatRequest):
    user_message = request.message

    # Add user input to history
    chat_history.append(HumanMessage(content=user_message))

    # Get model response
    result = llm.invoke(chat_history)

    # Append AI response
    chat_history.append(AIMessage(content=result.content))

    return ChatResponse(response=result.content)
